In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%run /PATH/packages/Basic.ipynb

In [ ]:
randomforest_classifier = RandomForestClassifier(n_estimators = 100, criterion = 'entropy')
datasets=["DS4","DS5","DS6","DS7","DS12","DS13","DS14","DS15"]
featureSelectionMethod={
      "randomForest":RandomForestClassifier(n_estimators=100),
      "XGBClassifier":XGBClassifier(),
}
outlierMethod={
        "iso":IsolationForest(contamination=0.1),
        "ee":EllipticEnvelope(contamination=0.01),
        "lof":LocalOutlierFactor(),
        "ncSVM":OneClassSVM(nu=0.01)
}

testDataSet={}
classifer_accuracy_result=pd.DataFrame(columns=["DS","Normalizer","FeatureSelected","Outlier","b-precision","b-recall","b-f1-score","b-total","h-precision","h-recall","h-f1-score","h-total","a-accuracy"])

for index,dataset in enumerate(datasets):
  testDataSet[dataset] = pd.read_csv(f'PATH/{dataset}.csv.gz',compression='gzip',lineterminator='\n');


for key,value in Normalizing(data).items():
  for keyOutlier,valueOutlier in outlierMethod.items():

    value= value[valueOutlier.fit_predict(value.drop(['class'],axis=1).values)!= -1]

    for keyFeature,valueFeature in featureSelectionMethod.items():

      if keyFeature=="all":
          selected_columns=value.columns.difference(["class"]).tolist()
      else: 
        selected_columns=returnSelectedFeature(valueFeature,value,15)

      randomforest_classifier.fit(value[selected_columns], value['class'])
      
      for keyTest,dataTest in testDataSet.items():

          normalizedTest=SingleNormalizing(dataTest,key).copy()
          classifer_accuracy_row=[keyTest,key,keyFeature,keyOutlier]


          for index_2,(keyShow,valueShow) in enumerate(classification_report(normalizedTest["class"], randomforest_classifier.predict(normalizedTest[selected_columns]),output_dict=True).items()):
            if keyShow in ["bot","human"]:
              classifer_accuracy_row.extend([*valueShow.values()])
            elif keyShow=="accuracy":
              classifer_accuracy_row.append(valueShow)
          print(key,keyFeature,keyTest,keyOutlier)
          classifer_accuracy_result.loc[len(classifer_accuracy_result)]=classifer_accuracy_row
classifer_accuracy_result.to_csv(f'PATH/DT_15_2022-06-02.csv.gz', compression='gzip',index=False)

In [ ]:
result= pd.read_csv(f'PATH/DT_15_2022-06-02.csv.gz',compression='gzip',lineterminator='\n')
result[(result['DS'].isin(["DS4","DS5","DS6","DS13"])) & (result['a-accuracy']>0.7)].sort_values(by=['a-accuracy'],ascending=False)

,DS,Normalizer,FeatureSelected,Outlier,b-precision,b-recall,b-f1-score,b-total,h-precision,h-recall,h-f1-score,h-total,a-accuracy
117,DS13,Normalizer,randomForest,ncSVM,0.578947,0.820896,0.679012,201,0.877966,0.683377,0.768546,379,0.731034
85,DS13,Normalizer,randomForest,ee,0.574468,0.805970,0.670807,201,0.869128,0.683377,0.765140,379,0.725862
101,DS13,Normalizer,randomForest,lof,0.565517,0.815920,0.668024,201,0.872414,0.667546,0.756353,379,0.718966
77,DS13,Normalizer,XGBClassifier,iso,0.565836,0.791045,0.659751,201,0.859532,0.678100,0.758112,379,0.717241
125,DS13,Normalizer,XGBClassifier,ncSVM,0.564014,0.810945,0.665306,201,0.869416,0.667546,0.755224,379,0.717241
69,DS13,Normalizer,randomForest,iso,0.562069,0.810945,0.663951,201,0.868966,0.664908,0.753363,379,0.715517
93,DS13,Normalizer,XGBClassifier,ee,0.562724,0.781095,0.654167,201,0.853821,0.678100,0.755882,379,0.713793
109,DS13,Normalizer,XGBClassifier,lof,0.561404,0.796020,0.658436,201,0.861017,0.670185,0.753709,379,0.713793


In [ ]:
result.groupby(["FeatureSelected","Normalizer"])['a-accuracy'].mean()

FeatureSelected  Normalizer    
XGBClassifier    Binarizer         0.676633
                 MaxAbsScaler      0.715713
                 MinMaxScaler      0.713440
                 Normalizer        0.760289
                 RobustScaler      0.546946
                 StandardScaler    0.479575
                 normal            0.658018
                 uniform           0.663779
                 yeoJohnson        0.606508
randomForest     Binarizer         0.663952
                 MaxAbsScaler      0.726747
                 MinMaxScaler      0.726591
                 Normalizer        0.756465
                 RobustScaler      0.546651
                 StandardScaler    0.489628
                 normal            0.651075
                 uniform           0.657295
                 yeoJohnson        0.607617
Name: a-accuracy, dtype: float64

In [ ]:
result[result['a-accuracy']>0.95]

In [ ]:
datasets=["DS4","DS5","DS6","DS7","DS12","DS13","DS14","DS15"]
featureSelectionMethod={
      "randomForest":RandomForestClassifier(n_estimators=100),
      "XGBClassifier":XGBClassifier(),
}
outlierMethod={
        "iso":IsolationForest(contamination=0.01),
        "ee":EllipticEnvelope(contamination=0.01),
        # "lof":LocalOutlierFactor(contamination=0.01),
        "ncSVM":OneClassSVM(gamma='scale', nu=0.01)
}

testDataSet={}
classifer_accuracy_result=pd.DataFrame(columns=["DS","Normalizer","FeatureSelected","Method","b-precision","b-recall","b-f1-score","b-total","h-precision","h-recall","h-f1-score","h-total","a-accuracy"])

for index,dataset in enumerate(datasets):
  testDataSet[dataset] = pd.read_csv(f'PATH/{dataset}.csv.gz',compression='gzip',lineterminator='\n');


for key,value in Normalizing(data).items():

  value=value[value["class"]=="human"]

  for keyFeature,valueFeature in featureSelectionMethod.items():
    if keyFeature=="all":
        selected_columns=value.columns.difference(["class"]).tolist()
    else: 
      selected_columns=returnSelectedFeature(valueFeature,value,15)

    for keyMethod,valueMethod in outlierMethod.items():

      valueMethod.fit(value[selected_columns].values)

      for keyTest,dataTest in testDataSet.items():

          normalizedTest=SingleNormalizing(dataTest,key).copy()
          normalizedTest['class'] = normalizedTest['class'].map({"bot": -1, "human": 1});

          classifer_accuracy_row=[keyTest,key,keyFeature,keyMethod]

          for index_2,(keyShow,valueShow) in enumerate(classification_report(normalizedTest["class"], valueMethod.predict(normalizedTest[selected_columns].values),output_dict=True).items()):
            if keyShow in ["-1","1"]:
              classifer_accuracy_row.extend([*valueShow.values()])
            elif keyShow=="accuracy":
              classifer_accuracy_row.append(valueShow)
          # print(key,keyFeature,keyTest,keyMethod,classifer_accuracy_row)
          print(classifer_accuracy_row)
          classifer_accuracy_result.loc[len(classifer_accuracy_result)]=classifer_accuracy_row
classifer_accuracy_result.to_csv(f'PATH/oneClass_15_2022-06-02.csv.gz', compression='gzip',index=False)